In [2]:
# # coding: utf-8
import pandas as pd
import numpy as np

import time

import data_load as dl

In [3]:
dl.get_data("""
select *
from tl.events

order by insert_time desc


limit 5


"""
)

,user_id,user_name,event_name,parameters,insert_time
0,249792088,rosolimo,button_click,{Button_name: Главное меню},2023-02-25 19:17:31.670108
1,249792088,rosolimo,button_click,{Button_name: Пишу текст},2023-02-25 18:50:00.299130
2,249792088,rosolimo,session_start,,2023-02-25 18:43:53.575158
3,249792088,rosolimo,button_click,{Button_name: Главное меню},2023-02-25 18:42:06.872350
4,249792088,rosolimo,button_click,{Button_name: Посмотреть мою индивидуальную статистику},2023-02-25 18:42:01.742182


In [5]:
dl.get_data("""
select distinct user_name
from tl.events



"""
)

,user_name
0,tsaregalina
1,rosolimo
2,noramornomore
3,eugeniabooo
4,tayistrash


In [6]:
dl.get_data("""
select *
from tl.events
where user_name = 'noramornomore'


"""
)

,user_id,user_name,event_name,parameters,insert_time
0,435209913,noramornomore,button_click,{Button_name: Main menu},2023-02-24 20:40:02.627563
1,435209913,noramornomore,button_click,{Button_name: Main menu},2023-02-24 20:46:45.010069
2,435209913,noramornomore,button_click,{Button_name: Main menu},2023-02-24 20:48:08.990450
3,435209913,noramornomore,session_start,,2023-02-24 20:55:49.735593
4,435209913,noramornomore,button_click,{Button_name: Рано},2023-02-24 20:55:49.734445
5,435209913,noramornomore,button_click,{Button_name: Main menu},2023-02-24 22:14:01.011276
6,435209913,noramornomore,session_start,,2023-02-25 16:37:14.187708
7,435209913,noramornomore,button_click,{Button_name: Main menu},2023-02-25 16:37:14.192390


In [9]:
dl.get_data("""
                    with
                    random_rate_old as 
                    (
                        select 
                        user_id, avg(result) as random_value, count(distinct insert_time) as attempts
                        from tl.game_results
                        group by user_id
                    ), 
                    left_tbl as 
                    (
                        select battle1 as item, user_id, "user"
                            , case 
                                when result = 0 then 1
                                else 0
                            end as is_selected
                                --, row_number() as id
                        from tl.game_results
                    ),
                    right_tbl as 
                    (
                            select battle2 as item, user_id, "user"
                                        , case 
                                            when result = 1 then 1
                                            else 0
                                        end as is_selected
                        from tl.game_results
                    ),
                    pre_items as 
                    (
                        select * from left_tbl

                        union all

                        select * from right_tbl
                    ),
                    items as 
                    (
                        select *, row_number() over() as id
                        from pre_items
                    ),
                    global_stat as 
                    (
                        select item
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item
                    ),
                    personal_stat as 
                    (
                        select item, user_id, "user"
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item, user_id, "user"
                    ),
                    indiv_stat as 
                    (
                        select ps.*
                                , gs.select_rate as global_select_rate
                                , case 
                                    when (ps.select_rate <= 0.05) or (ps.select_rate >= 0.95) then 0
                                    else  round((100 * abs(gs.select_rate - ps.select_rate))::decimal, 0)
                                end as original_score
                        from personal_stat ps
                        left outer join global_stat gs on 
                            (ps.item=gs.item)
                    ),
                    original as 
                    (
                        select user_id, "user"
                                , avg(original_score) as original_score, sum(total) / 2 as attemps
                        from  indiv_stat
                        group by user_id, "user"
                    ),
                    random_rate as 
                    (
                        select user_id, "user"
                                , stddev_pop(select_rate)
                                ,  case 
                                    when stddev_pop(select_rate) <= 0.1 then 0
                                    when stddev_pop(select_rate) >= 0.9 then 100
                                    else round((100*(stddev_pop(select_rate)))::decimal,0)
                                end as non_random_rate
                        from personal_stat
                        group by user_id, "user"
                    )


                    select 
                            o."user"
                            , round(((o.attemps * o.original_score) + (o.attemps * (50 - non_random_rate))) / 100,1) as score
                            , round(o.attemps,0) as attemps 
                            , round(o.original_score,0) as original_score
                            , non_random_rate
                    from original o
                    left outer join random_rate rr on
                        (rr.user_id = o.user_id)
                    order by 2 desc



                    """)

,user,score,attemps,original_score,non_random_rate
0,tsaregalina,21.4,130,4,38
1,rosolimo,19.1,112,8,41
2,vladchernichenko,8.7,25,16,31
3,eugeniabooo,2.4,21,4,43
4,noramornomore,0.6,3,11,41
5,tayistrash,0.0,3,0,49


In [17]:
dl.get_data("""
                    with
                    random_rate_old as 
                    (
                        select 
                        user_id, avg(result) as random_value, count(distinct insert_time) as attempts
                        from tl.game_results
                        group by user_id
                    ), 
                    left_tbl as 
                    (
                        select battle1 as item, user_id, "user"
                            , case 
                                when result = 0 then 1
                                else 0
                            end as is_selected
                                --, row_number() as id
                        from tl.game_results
                    ),
                    right_tbl as 
                    (
                            select battle2 as item, user_id, "user"
                                        , case 
                                            when result = 1 then 1
                                            else 0
                                        end as is_selected
                        from tl.game_results
                    ),
                    pre_items as 
                    (
                        select * from left_tbl

                        union all

                        select * from right_tbl
                    ),
                    items as 
                    (
                        select *, row_number() over() as id
                        from pre_items
                    ),
                    global_stat as 
                    (
                        select item
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item
                    ),
                    personal_stat as 
                    (
                        select item, user_id, "user"
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item, user_id, "user"
                    ),
                    indiv_stat as 
                    (
                        select ps.*
                                , gs.select_rate as global_select_rate
                                , case 
                                    when (ps.select_rate <= 0.05) or (ps.select_rate >= 0.95) then 0
                                    else  round((100 * abs(gs.select_rate - ps.select_rate))::decimal, 0)
                                end as original_score
                        from personal_stat ps
                        left outer join global_stat gs on 
                            (ps.item=gs.item)
                    ),
                    original as 
                    (
                        select user_id, "user"
                                , avg(original_score) as original_score, sum(total) / 2 as attemps
                        from  indiv_stat
                        group by user_id, "user"
                    ),
                    random_rate as 
                    (
                        select user_id, "user"
                                , stddev_pop(select_rate)
                                ,  case 
                                    when stddev_pop(select_rate) <= 0.1 then 0
                                    when stddev_pop(select_rate) >= 0.9 then 100
                                    else round((100*(stddev_pop(select_rate)))::decimal,0)
                                end as non_random_rate
                        from personal_stat
                        group by user_id, "user"
                    )


                    select *
                    from indiv_stat
                    where total > 2
                    and "user" = 'eugeniabooo'

                    order by original_score desc



                    """)

,item,user_id,user,selected,total,select_rate,global_select_rate,original_score
0,Герой нашего времени,1033768312,eugeniabooo,1,3,0.333333,0.461538,13
1,Война и мир,1033768312,eugeniabooo,2,3,0.666667,0.615385,5
2,Преступление и наказание,1033768312,eugeniabooo,5,5,1.000000,0.540541,0
3,Колобок,1033768312,eugeniabooo,0,4,0.000000,0.300000,0
4,Лабиринт отражений,1033768312,eugeniabooo,0,4,0.000000,0.214286,0
5,Мастер и Маргарита,1033768312,eugeniabooo,3,3,1.000000,0.921053,0


In [10]:
dl.get_data("""
                    with
                    random_rate_old as 
                    (
                        select 
                        user_id, avg(result) as random_value, count(distinct insert_time) as attempts
                        from tl.game_results
                        group by user_id
                    ), 
                    left_tbl as 
                    (
                        select battle1 as item, user_id, "user"
                            , case 
                                when result = 0 then 1
                                else 0
                            end as is_selected
                                --, row_number() as id
                        from tl.game_results
                    ),
                    right_tbl as 
                    (
                            select battle2 as item, user_id, "user"
                                        , case 
                                            when result = 1 then 1
                                            else 0
                                        end as is_selected
                        from tl.game_results
                    ),
                    pre_items as 
                    (
                        select * from left_tbl

                        union all

                        select * from right_tbl
                    ),
                    items as 
                    (
                        select *, row_number() over() as id
                        from pre_items
                    ),
                    global_stat as 
                    (
                        select item
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item
                    ),
                    personal_stat as 
                    (
                        select item, user_id, "user"
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item, user_id, "user"
                    ),
                    indiv_stat as 
                    (
                        select ps.*
                                , gs.select_rate as global_select_rate
                                , case 
                                    when (ps.select_rate <= 0.05) or (ps.select_rate >= 0.95) then 0
                                    else  round((100 * abs(gs.select_rate - ps.select_rate))::decimal, 0)
                                end as original_score
                        from personal_stat ps
                        left outer join global_stat gs on 
                            (ps.item=gs.item)
                    ),
                    original as 
                    (
                        select user_id, "user"
                                , avg(original_score) as original_score, sum(total) / 2 as attemps
                        from  indiv_stat
                        group by user_id, "user"
                    ),
                    random_rate as 
                    (
                        select user_id, "user"
                                , stddev_pop(select_rate)
                                ,  case 
                                    when stddev_pop(select_rate) <= 0.1 then 0
                                    when stddev_pop(select_rate) >= 0.9 then 100
                                    else round((100*(stddev_pop(select_rate)))::decimal,0)
                                end as non_random_rate
                        from personal_stat
                        group by user_id, "user"
                    )


                    select user_id, "user", item
                                , count(distinct id) as attempts
                                , round(avg(is_selected),2) as select_rate
                                , round(stddev_pop(is_selected),2) as std
                    from items
                    where 
                    "user" = 'eugeniabooo'

                    group by user_id, "user", item

                    order by 4 desc


                    """)

,user_id,user,item,attempts,select_rate,std
0,1033768312,eugeniabooo,Преступление и наказание,5,1.00,0.00
1,1033768312,eugeniabooo,Колобок,4,0.00,0.00
2,1033768312,eugeniabooo,Лабиринт отражений,4,0.00,0.00
3,1033768312,eugeniabooo,Война и мир,3,0.67,0.47
4,1033768312,eugeniabooo,Герой нашего времени,3,0.33,0.47
5,1033768312,eugeniabooo,Мастер и Маргарита,3,1.00,0.00
6,1033768312,eugeniabooo,Унесённые ветром,2,0.00,0.00
7,1033768312,eugeniabooo,Волкодав,2,1.00,0.00
8,1033768312,eugeniabooo,Джейн Эйр,2,0.50,0.50
9,1033768312,eugeniabooo,"Дядя Фёдор, пёс и кот",2,0.00,0.00
